In [1]:
pip install pandas scikit-learn nltk matplotlib ollama

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.7 MB/s eta 0:00:00


In [10]:
from google.colab import files
uploaded = files.upload()

Saving Professional-resume-Shirisha.pdf to Professional-resume-Shirisha.pdf


In [11]:
import PyPDF2

# Get uploaded filename
filename = list(uploaded.keys())[0]

with open(filename, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    resume_text = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            resume_text += text

print("Resume loaded successfully ✅")

Resume loaded successfully ✅


In [12]:
job_description = """
We are hiring a Dotnet Developer Engineer with strong C# skills,
ADO.NET experience, SQL knowledge and experience with Azure.
"""

In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

In [14]:
def compute_similarity(resume, job_desc):

    documents = [
        clean_text(resume),
        clean_text(job_desc)
    ]

    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)

    similarity = cosine_similarity(
        tfidf_matrix[0:1],
        tfidf_matrix[1:2]
    )

    return round(float(similarity[0][0]) * 100, 2)

In [15]:
def extract_keywords(text):
    vectorizer = TfidfVectorizer(
        stop_words='english',
        max_features=30
    )

    tfidf_matrix = vectorizer.fit_transform([clean_text(text)])

    return set(vectorizer.get_feature_names_out())


def skill_gap_analysis(resume, job_desc):

    resume_keywords = extract_keywords(resume)
    job_keywords = extract_keywords(job_desc)

    matched_skills = resume_keywords.intersection(job_keywords)
    missing_skills = job_keywords - resume_keywords

    return matched_skills, missing_skills

In [16]:
similarity_score = compute_similarity(
    resume_text,
    job_description
)

matched_skills, missing_skills = skill_gap_analysis(
    resume_text,
    job_description
)

print("Match Score:", similarity_score, "%")
print("\nMatched Skills:", matched_skills)
print("\nMissing Skills:", missing_skills)

Match Score: 16.26 %

Matched Skills: {'developer', 'experience', 'sql', 'skills'}

Missing Skills: {'adonet', 'hiring', 'engineer', 'dotnet', 'strong', 'azure', 'knowledge'}


In [17]:
#LLM feedback generation

In [18]:
!pip install transformers torch accelerate

In [20]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="google/flan-t5-base",
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausa

In [25]:
def generate_resume_feedback(resume, job_desc, similarity, missing_skills):

    prompt = f"""
Resume:
{resume}

Job Description:
{job_desc}

Match Score: 16.26%

Missing Skills: {'adonet', 'hiring', 'engineer', 'dotnet', 'strong', 'azure', 'knowledge'}
"""
    response = generator(
        prompt,
        max_length=512,
        do_sample=True,
        temperature=0.7
    )

    return response[0]["generated_text"]

In [26]:
feedback = generate_resume_feedback(
    resume_text,
    job_description,
    similarity_score,
    missing_skills
)

print(feedback)

Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Resume:
ACCENTURE | 09/2024 – 09/25  
 
Metrolinx  (Fare Management System) - CANADA  
Team Size: 30 (L3 Support) | Program Size: 300+  
 
Responsibilities & Impacts : 
• Provided production and technical support for a fare management system 
used by millions of daily commuters  across Ontario transit networks.  
• Technology used: C#, ASP.NET 4.8 x, SQL  Server , Azure DevOps, React JS, 
Postman, Event Viewer  
• Resolved 15 –20 production incidents per month, reducing ticket backlog by 
30% and improving SLA adherence to 98%. 
• Performed root cause analysis using Event Viewer, logs, and SQL tracing, 
reducing recurring defects by 35%. 
• Developed and optimized SQL stored procedures, views, joins, and triggers, 
improving data retrieval performance by 30%. 
• Experience on migration on old technology à new technology.  
• Expe rience on Agile methodologies, c onduct code reviews and manage the 
process. Demonstrated expertise in software requirements and designed use 
case diagrams